In [1]:
# given a city and business category, how far is a customer willing to travel

In [2]:
from pymongo import MongoClient
import math
from geopy.distance import vincenty
from tqdm import tqdm
import json
client = MongoClient()
db = client.yelp

In [18]:
# list businesses by city
def businesses_by_city(city_name):
    business_by_city_list = []
    for x in db.businesses.find({"city": city_name}):
        business_by_city_list.append(x["business_id"])
    return business_by_city_list

In [20]:
print(len(businesses_by_city("Tempe")))

3703


In [42]:
# get list of industries by city
def industry_list_by_city(city_name):
    industry_list = []
    for x in db.businesses.find({"city": city_name}):
        sub_industry_list = x["categories"]
        if sub_industry_list:
            for z in sub_industry_list:
                if z not in industry_list:
                    industry_list.append(z)
    return industry_list

In [384]:
# print(len(industry_list_by_city("Tempe")))
print(industry_list_by_city("Tempe"))

[u'Tobacco Shops', u'Nightlife', u'Vape Shops', u'Shopping', u'Active Life', u'Trainers', u'Fitness & Instruction', u'Interior Design', u'Home Services', u'Shades & Blinds', u'Fast Food', u'Restaurants', u'Food', u'Coffee & Tea', u'Opera & Ballet', u'Education', u'Performing Arts', u'Dance Schools', u'Specialty Schools', u'Arts & Entertainment', u'Health & Medical', u'Naturopathic/Holistic', u'Doctors', u'Beauty & Spas', u'Nail Salons', u'Cards & Stationery', u'Flowers & Gifts', u'Arts & Crafts', u'Event Planning & Services', u'Yoga', u'Barre Classes', u'Pilates', u'Employment Agencies', u'Professional Services', u'Fashion', u'Lingerie', u'Tutoring Centers', u'Test Preparation', u'Summer Camps', u'Pakistani', u'Halal', u'Indian', u'Skin Care', u'Medical Spas', u'Hair Removal', u'Laser Hair Removal', u'Day Spas', u'Keys & Locksmiths', u'Bikes', u'Sporting Goods', u'Sports Wear', u'Outdoor Gear', u'Party Equipment Rentals', u'Party Supplies', u'Local Services', u'Hair Extensions', u'Hair

In [65]:
# print(len(get_businesses_from_industry_and_city("Tempe", "Emergency Rooms")))
# get_businesses_from_industry_and_city("Tempe", "Emergency Rooms")

# get all businesses in a city, .distinct(categories)
def list_categories_by_city(city):
    return db.businesses.find({"city": city}).distinct("categories")

In [69]:
print(len(list_categories_by_city("Tempe")))

740


In [174]:
# loop through industry list and pass in one industry at a time
# given a city and industry, get all businesses
def get_businesses_from_industry_and_city(city):
#     businesses = businesses_by_city(city)
#     businesses = db.businesses.find({"city": city}) #.distinct("business_id")
#     print(businesses)
#     print(type(businesses))
    return db.businesses.find({"business_id": {"$elemMatch": {"$in": businesses}}, "categories": {"$elemMatch": {"$in": industries}}})


In [5]:
# 3) separate businesses by user's that have reviewed them
# def get_businesses_user_reviewed(user):
#     # get all reviews that contain a previously calculated business_id by 
#     user_reviewed_businesses = db.reviews.find_one("business_id": )
#     return user_reviewed_businesses



def get_users_by_business_reviews(bCategory, newKey):
    print "users by reviews"
    # already found all users
#     filtered_reviews = db.reviews.find({"business_id": {"$in": bCategory.distinct("business_id")}})
#     users = filtered_reviews.distinct("user_id")
    
    print("user_location: ",user_location)
    for user in tqdm(user_location):
        
        category_reviews_by_user = db.reviews.find({"user_id": user, "business_id": {"$in": bCategory.distinct("business_id")}})
        short_b = category_reviews_by_user.distinct("business_id")
        
        home = (user_location[user][0], user_location[user][1])
        
#         latitude = db.businesses.find({"business_id": {"$in": short_b}}).distinct("latitude")
#         longitude = db.businesses.find({"business_id": {"$in": short_b}}).distinct("longitude")

        short_b2 = db.businesses.find({"business_id": {"$in": short_b}})
        
#         for i in range(len(latitude)):
        for b in short_b2:
#             print b['latitude'], b['longitude']
#             lat, lon = float(latitude[i]), float(longitude[i])
#             business_loc = (float(latitude[i]), float(longitude[i]))
            business_loc = (float(b['latitude']), float(b['longitude']))
            miles_dist = vincenty(home, business_loc).miles
        
            temp = category_ave_distance_by_city[newKey]
            category_ave_distance_by_city[newKey] = (temp[0] + miles_dist, temp[1] + 1)
            
#             temp = category_ave_distance_by_city[category_ave_distance_by_city["lastCity"]][category]
#             category_ave_distance_by_city[category_ave_distance_by_city["lastCity"]][category] = [temp[0] + miles_dist, temp[1] + 1]
#         print(len(list(new_list)))
        break
    
#     print(len(users))
#     print(len(list(filtered_reviews)))
#     print(len(list(bCategory)))


In [6]:
user_location = {} # empty dictionary
category_ave_distance_by_city = {}

# def calc_living_location(user, reviews):
#     # again, why?: AttributeError: 'Cursor' object has no attribute 'find'
#     # businesses_in_city_reviewed_by_user = reviews.find({"user_id": user}).distinct("business_id")
# #     businesses_in_city_reviewed_by_user = reviews.find({})
#     latitude = 0.0
#     longitude = 0.0
#     count = len(businesses_in_city_reviewed_by_user)
#     business_obj_in_city_reviewed_by_user = db.businesses.find({"business_id": {"$in": businesses_in_city_reviewed_by_user}})
#     for b in business_obj_in_city_reviewed_by_user:
#         latitude += float(b['latitude'])
#         longitude += float(b['longitude'])
    
#     if count > 8:
#         user_location[user] = [latitude/count, longitude/count]

def get_user_living_location(businesses_in_specified_city):
    all_reviews_by_users = db.reviews.find({"business_id": {"$in": businesses_in_specified_city}})
    users_in_city = all_reviews_by_users.distinct("user_id")
    
    for user in tqdm(users_in_city):
        businesses_in_city_reviewed_by_user = db.reviews.find({"business_id": {"$in": businesses_in_specified_city}, "user_id": user}).distinct("business_id")
        latitude = 0.0
        longitude = 0.0
        count = len(businesses_in_city_reviewed_by_user)
        business_obj_in_city_reviewed_by_user = db.businesses.find({"business_id": {"$in": businesses_in_city_reviewed_by_user}})
        for b in business_obj_in_city_reviewed_by_user:
            latitude += float(b['latitude'])
            longitude += float(b['longitude'])


        if count > 8:
            user_location[user] = [latitude/count, longitude/count]
        else:
            continue
#         print "businesses reviewed in city: ",count
#         print "reviews total: ", len(list(db.reviews.find({"business_id": {"$in": businesses_in_specified_city}, "user_id": user}).distinct("review_id")))
#         print (user_location[user])    
#         break
#         print user
#         print users_in_city
#         single_user = ["{}".format(user)]
#         print user in all_reviews_by_users.distinct("user_id")
#         reviews_by_one_user = db.reviews.find({"business_id": {"$in": businesses_in_specified_city}, "user_id": user})
    
        # why?
#         reviews_by_one_user = all_reviews_by_users.find({"user_id": user}) # error: AttributeError: 'Cursor' object has no attribute 'find'
#         calc_living_location(user, reviews_by_one_user)
#         break

In [ ]:
item = db.userAddress.find().distinct('city')
print len(item)

In [39]:
def get_users_by_reviews(bCategory, newKey, city, users_in_city):
#     users_in_city = db.userAddress.find({'city': city}).distinct('user_id')
    for user in users_in_city:
        category_reviews_by_user = db.reviews.find({"user_id": user, "business_id": {"$in": bCategory.distinct("business_id")}})
        short_b = category_reviews_by_user.distinct("business_id")
        
#         print "city: ", city
#         print 'user: ', user
        item = db.userAddress.find_one({'city': city, 'user_id': user})
#         print item
#         print city, user
        home = item['living_location'] 
        # (user_location[user][0], user_location[user][1])

        short_b2 = db.businesses.find({"business_id": {"$in": short_b}})
        for b in short_b2:
            business_loc = [float(b['latitude']), float(b['longitude'])]
            miles_dist = vincenty(home, business_loc).miles
        
            temp = category_ave_distance_by_city[newKey]
            category_ave_distance_by_city[newKey] = (temp[0] + miles_dist, temp[1] + 1)

In [ ]:
category_ave_distance_by_city = {}
cities = db.userAddress.find().distinct('city')
for city in tqdm(cities):
    users_in_city = db.userAddress.find({'city': city}).distinct('user_id')
    industries = db.businesses.find({"city": city}).distinct("categories")
    businesses = db.businesses.find({"city": city}).distinct("business_id")
    for industry in industries:
        category = ["{}".format(industry)]
        businesses_by_category = db.businesses.find({"business_id": {"$in": businesses}, "categories": {"$elemMatch": {"$in": category}}})
        size = len(businesses_by_category.distinct("business_id"))
        if size > 8:
            newKey = city + industry
            category_ave_distance_by_city[newKey] = (0.0, 0.0)
            get_users_by_reviews(businesses_by_category, newKey, city, users_in_city)
        else:
            continue

print "{:25}{}".format("items in category: ",category_ave_distance_by_city[newKey][1])
for c in category_ave_distance_by_city:
    temp = category_ave_distance_by_city[c]
    if temp[1] != 0:
        category_ave_distance_by_city[c] = (temp[0]/temp[1], 1)

print "{:25}{}".format(newKey, category_ave_distance_by_city[newKey])

 15%|█▍        | 27/185 [09:33<6:09:41, 140.39s/it]

In [7]:
city = "Tempe"
# category_ave_distance_by_city["lastCity"] = city
industries = db.businesses.find({"city": city}).distinct("categories")
businesses = db.businesses.find({"city": city}).distinct("business_id")

# print "5jxn3_C5rjBow-aiBqcQNg" in businesses

print(len(industries))
print(len(businesses))

# print industries

# now that it's in a database, no need to call this tedius function anymore for the city "Tempe"
# get_user_living_location(businesses)

for industry in industries:
    # if True:
    category = ["{}".format(industry)]
#         print category
    
    businesses_by_category = db.businesses.find({"business_id": {"$in": businesses}, "categories": {"$elemMatch": {"$in": category}}})

    # wrong way of getting the size:
    size = len(businesses_by_category.distinct("business_id"))
    if size > 8:
#         catDict = {}
        newKey = city + industry
#         catDict[industry] = [0.0, 0.0]
        category_ave_distance_by_city[newKey] = (0.0, 0.0)
        get_users_by_business_reviews(businesses_by_category, newKey)
    else:
        continue
#     print(industry, list(businesses_by_category))
#     print(len(list(businesses_by_category)))
    
    
#     print "category {}".format(newKey), category_ave_distance_by_city[newKey]
#     break
    #     calculate get_businesses_user_reivewed()

    
    # last step!!!!!!!!!
    # average the distances for the given city:
    for c in category_ave_distance_by_city:
        temp = category_ave_distance_by_city[c]
        if temp[1] != 0:
            category_ave_distance_by_city[c] = (temp[0]/temp[1], 1)
        
    print "category {}".format(newKey), category_ave_distance_by_city[newKey]
    break
    
# print len(businesses)
# print len(list(db.businesses.find({"business_id": {"$in": businesses}})))
# print list(db.businesses.find({"city": city, "categories": {"$elemMatch": {"$in": category}}}))
# for x in businesses_by_category:
#     print(x)

# get_businesses_from_industry_and_city("Tempe", "Business Law")

740
3703


  0%|          | 0/1882 [00:00<?, ?it/s]

users by reviews
('user_location: ', {u'VxaNYhSruuFLfbTPs99ITg': [33.38102177426316, -111.93069285778945], u'nUlh8GQ1vHGpfeB6It3bvw': [33.422641897587496, -111.93415336056249], u'6bIw0iBMzJsw8uOrR92QaQ': [33.41379668986, -111.93987634329999], u'mcfvIMIxf21H5g2pTbushg': [33.406361789554545, -111.92194398218182], u'6uREWrcg14EdD3UXvmPp0g': [33.377390379015, -111.935692880175], u'tg4YtJdtO0RDskJEjcBvNA': [33.41058741214444, -111.94794870133333], u'9OnR3G7njmSxcJ_7WirxoA': [33.399535923618195, -111.93561613645457], u'N-wK1EpQs-TReteAH-WB9Q': [33.40687330984545, -111.93167542781818], u'W47nK4IckxZ65KJuIjXvRg': [33.40659085608, -111.92358124700002], u'_jjbwHWLFA5GjWFWGLj4ZQ': [33.395941364815506, -111.93535935339654], u'Tr7YMpWd1XZdXdm35CatqA': [33.399746700373335, -111.93103350093332], u'kjeX2RXvW7RhBbD2QLd5jA': [33.41873948618275, -111.92481947799999], u'tmpUupKfrWA6CxhgxdYhUA': [33.41622058361667, -111.92654076250001], u'94HchNwPr69BdrF0On9uKQ': [33.39739283183, -111.92313879024996], u'oE

In [22]:
# get copy of "Tempe" data
user_location = {}
tempe_data = db.userAddress.find({"city": "Tempe"})
for item in tempe_data:
    print item
    user_location[item["user_id"]] = item["living_location"]

{u'living_location': [33.38102177426316, -111.93069285778945], u'city': u'Tempe', u'_id': ObjectId('58c72b6dca05273287a8bac8'), u'user_id': u'VxaNYhSruuFLfbTPs99ITg'}
{u'living_location': [33.422641897587496, -111.93415336056249], u'city': u'Tempe', u'_id': ObjectId('58c72b6dca05273287a8bac9'), u'user_id': u'nUlh8GQ1vHGpfeB6It3bvw'}
{u'living_location': [33.41379668986, -111.93987634329999], u'city': u'Tempe', u'_id': ObjectId('58c72b6dca05273287a8baca'), u'user_id': u'6bIw0iBMzJsw8uOrR92QaQ'}
{u'living_location': [33.406361789554545, -111.92194398218182], u'city': u'Tempe', u'_id': ObjectId('58c72b6dca05273287a8bacb'), u'user_id': u'mcfvIMIxf21H5g2pTbushg'}
{u'living_location': [33.377390379015, -111.935692880175], u'city': u'Tempe', u'_id': ObjectId('58c72b6dca05273287a8bacc'), u'user_id': u'6uREWrcg14EdD3UXvmPp0g'}
{u'living_location': [33.41058741214444, -111.94794870133333], u'city': u'Tempe', u'_id': ObjectId('58c72b6dca05273287a8bacd'), u'user_id': u'tg4YtJdtO0RDskJEjcBvNA'}
{u'

In [28]:
data = db.userAddress.find({"city":"Tempe"})
print len(list(data))

1882


In [30]:
# import all cities to the database
# run exactly once
cities = db.businesses.find().distinct("city")

for city in cities:
    businesses = db.businesses.find({"city": city}).distinct("business_id")
    user_location = {}
    get_user_living_location(businesses)
    print city, len(user_location)

    for user in tqdm(user_location):
        entry = {'city': city, 'user_id': user, "living_location": user_location[user]}
        db.userAddress.insert_one(entry)


100%|██████████| 5/5 [00:00<00:00, 922.15it/s]
0it [00:00, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 1174.88it/s]
0it [00:00, ?it/s]

100%|██████████| 4/4 [00:00<00:00, 1089.01it/s]
0it [00:00, ?it/s]

100%|██████████| 4/4 [00:00<00:00, 1348.65it/s]
0it [00:00, ?it/s]

100%|██████████| 12/12 [00:00<00:00, 1130.16it/s]
0it [00:00, ?it/s]

  7%|▋         | 50/669 [00:00<00:01, 499.81it/s]

 0
AGINCOURT 0
Aberdour 0
Aberlady 0
Ahwahtukee 0



100%|██████████| 669/669 [00:01<00:00, 520.55it/s]
0it [00:00, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 1141.15it/s]
0it [00:00, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 877.98it/s]
0it [00:00, ?it/s]

  2%|▏         | 20/972 [00:00<00:04, 195.62it/s]

Ahwatukee 0
Ahwatukee Foothills Village 0
Aichwald 0



100%|██████████| 4/4 [00:00<00:00, 1254.00it/s]
0it [00:00, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 1324.13it/s]
0it [00:00, ?it/s]

  8%|▊         | 45/541 [00:00<00:01, 448.61it/s]

...

York 0


100%|██████████| 443/443 [00:00<00:00, 683.44it/s]
0it [00:00, ?it/s]
100%|██████████| 34/34 [00:00<00:00, 664.62it/s]
0it [00:00, ?it/s]
100%|██████████| 7/7 [00:00<00:00, 713.13it/s]
0it [00:00, ?it/s]
100%|██████████| 15/15 [00:00<00:00, 689.34it/s]
0it [00:00, ?it/s]
  8%|▊         | 72/902 [00:00<00:01, 719.49it/s]

Youngtown 0
columbia station 0
etobicoke 0
frazer 0


100%|██████████| 902/902 [00:01<00:00, 691.52it/s]
0it [00:00, ?it/s]
100%|██████████| 33/33 [00:00<00:00, 721.60it/s]
0it [00:00, ?it/s]
100%|██████████| 31/31 [00:00<00:00, 714.72it/s]
0it [00:00, ?it/s]
100%|██████████| 3/3 [00:00<00:00, 657.04it/s]
0it [00:00, ?it/s]
100%|██████████| 10/10 [00:00<00:00, 693.43it/s]
0it [00:00, ?it/s]

las vegas 0
mesa 0
nboulder city 0
oakville 0
pleasant hills 0


In [25]:
for user in tqdm(user_location):
    entry = {'city': 'Tempe', 'user_id': user, "living_location": user_location[user]}
    db.userAddress.insert_one(entry)


100%|██████████| 1882/1882 [00:00<00:00, 4048.46it/s]

In [19]:
print(list(db.userAddress.find()))

[{u'living_location': [33.38102177426316, -111.93069285778945], u'city': u'Tempe', u'_id': ObjectId('58c72b6dca05273287a8bac8'), u'user_id': u'VxaNYhSruuFLfbTPs99ITg'}, {u'living_location': [33.422641897587496, -111.93415336056249], u'city': u'Tempe', u'_id': ObjectId('58c72b6dca05273287a8bac9'), u'user_id': u'nUlh8GQ1vHGpfeB6It3bvw'}, {u'living_location': [33.41379668986, -111.93987634329999], u'city': u'Tempe', u'_id': ObjectId('58c72b6dca05273287a8baca'), u'user_id': u'6bIw0iBMzJsw8uOrR92QaQ'}, {u'living_location': [33.406361789554545, -111.92194398218182], u'city': u'Tempe', u'_id': ObjectId('58c72b6dca05273287a8bacb'), u'user_id': u'mcfvIMIxf21H5g2pTbushg'}, {u'living_location': [33.377390379015, -111.935692880175], u'city': u'Tempe', u'_id': ObjectId('58c72b6dca05273287a8bacc'), u'user_id': u'6uREWrcg14EdD3UXvmPp0g'}, {u'living_location': [33.41058741214444, -111.94794870133333], u'city': u'Tempe', u'_id': ObjectId('58c72b6dca05273287a8bacd'), u'user_id': u'tg4YtJdtO0RDskJEjcBvNA

In [189]:
# 4) count how many locations of the same industry a user has reviewed (count of #3)

In [190]:
# 5) if count < 2, ignore, else user lat address = ∑lat/n, user lon address = ∑lon/n, where n is step 4
    # note: can do this because lat/lon doesn't change much propotionally in the same city

In [191]:
# 6) average list of distances from user address to #3 of that user --> convert to miles (or just convert to miles in step 5)

In [ ]:
# calc distance in miles example
# newport_ri = (41.49008, -71.312796)
# cleveland_oh = (41.499498, -81.695391)
# print(vincenty(newport_ri, cleveland_oh).miles)

# cursor.rewind()
# db.collection.insert_one({JSON HERE})